# strategy

> Fill in a module description here

In [ ]:
#| default_exp strategy

In [ ]:
#| hide
from nbdev.showdoc import *

ModuleNotFoundError: No module named 'nbdev'

In [1]:
%load_ext autoreload


In [ ]:
%autoreload 2

In [ ]:
#| export
from clue.core import createClue,printLocs,printHand,keyLookUp,cardFull
from clue.core import getPlay,relocatePlayers,showChoice,getAnswer,getSeen
from clue.core import keysForGuess,printGame, keyCard, cardLookup
from clue.core import people ,weapons , rooms
from random import choice,shuffle

In [ ]:
#| hide
agame = createClue(4)
amoves = []
player = 1
done = False
printLocs(agame) 
done = False

shuffle(rooms)
sh = agame[1]

printHand([keyLookUp(x) for x in sh],cardFull)

[1] 👠 Miss Scarlet is in Dining Room
[2] 🧐 Colonel Mustard is in Lounge
[3] ♡  Mrs. White is in Ball Room
[4] 🐉 Mr. Green is in Kitchen
[5] 🍆 Professor Plum is in Conservatory
[6] 🦚 Mrs. Peacock is in Billiard Room
🔪 Knife, 👠 Miss Scarlet, 🔧 Wrench, 🦚 Mrs. Peacock, ♡  Mrs. White


In [ ]:
import tabulate
from termcolor import colored
import os

def playerMarker(i):
    info = people[i]
    color,icon, name = info[1:]
    #print(color)
    if color == "white":
        return colored("W",color)
    return colored(name[0],color)
   
    
def roomFor(i,ch,where):
    #print(i)
 
    if i<1:
        return ""
    adds = ""
    
    peeps = ", ".join([playerMarker(x) for x in where[i-1]])

    try:
        x = ch.index(i-1)
        adds = str(x+1) +") "
    except:
        adds = ""
        #print(i)
    return adds + rooms[i-1][0] +"\n   " + peeps

def showBoard(pc,game,moves):
    where = [[] for x in range(len(rooms))]
    ploc = game[-1]
    #print(ploc)
    for i in range(len(ploc)):
        where[ploc[i-1]].append(i)
    #print(where)
    
    exclude  = getSeen(pc,game, moves)
    
    lines = [[1,2,3],[9,0,4],[8,0,5],[0,7,6]]
    r2 = location(pc,game)
    b = trips(r2,len(rooms))
    m = []
    #print(b)
    for x in lines:
        m.append([roomFor(i,b,where) for i in x])
    print(tabulate.tabulate(m,  tablefmt="fancy_grid"))
    
showBoard(1,agame,[])
printLocs(agame) 
print(rooms)

#print(playerMarker(1))

╒══════════════════╤══════════════╤══════════════╕
│ 1) Billiard Room │ 2) Ball Room │ 3) Lounge    │
│    G             │    S         │              │
├──────────────────┼──────────────┼──────────────┤
│ Dining Room      │              │ Study        │
│    P             │              │              │
├──────────────────┼──────────────┼──────────────┤
│ Kitchen          │              │ Hall         │
│    P             │              │    M         │
├──────────────────┼──────────────┼──────────────┤
│                  │ Library      │ Conservatory │
│                  │    W         │              │
╘══════════════════╧══════════════╧══════════════╛
[1] 👠 Miss Scarlet is in Ball Room
[2] 🧐 Colonel Mustard is in Hall
[3] ♡  Mrs. White is in Library
[4] 🐉 Mr. Green is in Billiard Room
[5] 🍆 Professor Plum is in Dining Room
[6] 🦚 Mrs. Peacock is in Kitchen
[['Billiard Room', 'grey', '', 'Bill'], ['Ball Room', 'grey', '', 'Ball'], ['Lounge', 'grey', '', 'Lounge'], ['Study', 'grey', ''

In [ ]:
#| export
def location(playerID,game):
    ploc = game[-1]
    return ploc[playerID-2]

def trips(x,l):
    r1 = x -1
    if r1 < 0:
        r1 = l - 1
    r3 = x + 1
    if r3 > l - 1:
        r3 = 0
    return [r1,x,r3]

def seenCounts(pc,game, moves):
    knoweledge = getSeen(pc,game, moves)
    p = [0,0,0]
    for x in knoweledge:
        i = int(x/100) -1
        p[i] += 1

    return p

def simpleGuess(pc,game, moves):
    

    r2 = location(pc,game)
    exclude  = set(getSeen(pc,game, moves))
    
    pdeck = [keyCard(1,x) for x in range(len(people))]
    pdeck = [x for x in  pdeck if x not in exclude]
    p = choice(pdeck)%100
    
    pdeck = [keyCard(2,x) for x in range(len(weapons))]
    pdeck = [x for x in pdeck if x not in exclude]
    w = choice(pdeck)%100
    
    pdeck = [keyCard(3,x) for x in trips(r2,len(rooms))]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)


In [ ]:
#| export
def playerName(i):
    return cardFull(cardLookup(1,i-1))
#playerName(1)

In [ ]:
#| hide
printHand([keyLookUp(x) for x in sh],cardFull)

strategies = [simpleGuess for x in range(4)]
while not done:
    #print(seenCounts(player,agame,amoves))
    if player == 1:
        showBoard(player,agame,amoves)
        #printLocs(agame) 
        ans = getPlay(player,agame,amoves)
        #os.system('clear')
    else:
        strat = strategies[player-1]
        ans = strat(player,agame,amoves)
        
    
    agame = relocatePlayers(player , ans[0],ans[2],agame)
    print( playerName(player) + " suspects " + showChoice(ans[0],ans[1],ans[2])+".")
    check = getAnswer(player,agame,ans)
    if check[0] < 0:
        seen =  getSeen(player,agame, amoves)
        exclude  = set(seen)
        done = True
        for x in keysForGuess(ans[0],ans[1],ans[2]):
            if x in exclude:
                print("\n\n gave a false one " +  cardFull(keyLookUp(x))+"!")
                done = False
        
    else:
        #(p,w,l,playerNum, c,y[0],y[1])
        card = check[-1]
        
        amoves.append(check)
        if player == 1:
            print( playerName(check[-2]) + " has " + cardFull(keyLookUp(card))+"!")
        else:
            print( playerName(check[-2]) + " responds.")
        #printLocs(agame)
        player = player + 1
        if player>=len(agame)-1:
           player = 1 

print("\n" + playerName(player) + " won!")
printGame(agame) 

🔪 Knife, 👠 Miss Scarlet, 🔧 Wrench, 🦚 Mrs. Peacock, ♡  Mrs. White
╒═════════════════╤═══════════════╤═════════════╕
│ 3) Study        │ Library       │ Dining Room │
│    P            │    W          │    M        │
├─────────────────┼───────────────┼─────────────┤
│ 2) Hall         │               │ Lounge      │
│    S            │               │             │
├─────────────────┼───────────────┼─────────────┤
│ 1) Conservatory │               │ Ball Room   │
│    G            │               │             │
├─────────────────┼───────────────┼─────────────┤
│                 │ Billiard Room │ Kitchen     │
│                 │               │    P        │
╘═════════════════╧═══════════════╧═════════════╛

           [1] 👠 Miss Scarlet is in Hall            

Suspect
1)* 👠 Miss Scarlet
2)  🧐 Colonel Mustard
3)* ♡  Mrs. White
4)  🐉 Mr. Green
5)  🍆 Professor Plum
6)* 🦚 Mrs. Peacock


Choose from 1 to 6:  2




Weapon
1)* 🔪 Knife
2)  🔫 Revolver
3)  🪢 Rope
4)* 🔧 Wrench
5)  🕯️ Candlestick
6)  🪠 Lead Pipe


Choose from 1 to 6:  2




Move To
1)  Conservatory
2)  Hall
3)  Study


...

Choose from 1 to 6:  6




Move To
1)  Hall
2)* Study
3)  Library


Choose from 1 to 3:  1


👠 Miss Scarlet suspects 🍆 Professor Plum with the 🪠 Lead Pipe in the Hall.
🧐 Colonel Mustard has Hall!
🧐 Colonel Mustard suspects 🧐 Colonel Mustard with the 🪢 Rope in the Hall.


 gave a false one Hall!
🧐 Colonel Mustard suspects 🦚 Mrs. Peacock with the 🔫 Revolver in the Conservatory.
♡  Mrs. White responds.
♡  Mrs. White suspects 🐉 Mr. Green with the 🕯️ Candlestick in the Conservatory.
🐉 Mr. Green responds.
🐉 Mr. Green suspects 🧐 Colonel Mustard with the 🪢 Rope in the Billiard Room.
♡  Mrs. White responds.
╒═════════════════╤═══════════════╤═════════════╕
│ 3) Study        │ Library       │ Dining Room │
├─────────────────┼───────────────┼─────────────┤
│ 2) Hall         │               │ Lounge      │
│    S, P         │               │             │
├─────────────────┼───────────────┼─────────────┤
│ 1) Conservatory │               │ Ball Room   │
│    W, P         │               │             │
├─────────────────┼───────────────┼─────────────┤
│                 │ Billiard Room │ K

KeyboardInterrupt: 

In [ ]:
#| export
weapons = []
weapons.append(["Knife","cyan",u"\U0001F52A","Knife"])
weapons.append(["Revolver","cyan",u"\U0001F52B","Gun"])
weapons.append(["Rope","cyan",emoji.emojize(":knot:"),"Rope"])
weapons.append(["Wrench","cyan",emoji.emojize(":wrench:"),"Wrench"])
weapons.append(["Candlestick","cyan",emoji.emojize(":candle:"),"Candle"])
weapons.append(["Lead Pipe","cyan",emoji.emojize(":plunger:"),"Pipe"])

In [ ]:
#| export
rooms = []
rooms.append(["Ball Room","grey","","Ball"])
rooms.append(["Billiard Room","grey","","Bill"])
rooms.append(["Conservatory","grey","","Consv"])
rooms.append(["Dining Room","grey","","Dining"])
rooms.append(["Hall","grey","","Hall"])
rooms.append(["Kitchen","grey","","Kitch"])
rooms.append(["Lounge","grey","","Lounge"])
rooms.append(["Library","grey","","Lib"])
rooms.append(["Study","grey","","Study"])

In [ ]:
#| export
def cIndentity(x):
  return x[0]
    
def printHand(hand,view=cIndentity):
    container = []
    line = [view(card) for card in hand]
    print(', '.join(line))


In [ ]:
#| hide
printHand(people)

In [ ]:
#| hide
printHand(people,cardFull)
printHand(people,cardShort)

In [ ]:
#| export
def keyForCard(v):
    return v[0] * 100 + v[1]
    
def chooseCard(hand,view=cIndentity,fmt="fancy_grid",flag = {}):
    container = []
    i = 1
    ml = max([len(x[0]) for x in hand])

    no = " "
    
    for card in hand:
        face = view(card)
        #face.append(i)
        color = card[1]
        no = " "
        if card[0] in flag:
            no = "*"
        container.append(colored(str(i) +")" + no + " " ,color) + face)
        i += 1
    choice = -1
    
    while choice < 1:
        for card in container:
            print(card)
        choice = int(input("Choose from 1 to " +  str(len(container))+": "))
       
        if (choice > 0) and (choice <= len(container)):
            return choice
        print(str(choice) + " is not in range")
        choice = -1
            



In [ ]:
#| hide
chooseCard(people,cardFull,flag = {"Mr. Green"})

In [ ]:
#| hide
chooseCard(people,cardShort,flag = {"Mr. Green"})

In [ ]:
#| hide
print(u"\U0001F52A",u"\U0001F52B",emoji.emojize(":wrench:"),emoji.emojize(":candle:"),u"\U0001F99A")

In [ ]:
#| hide
chooseCard(weapons,cardFull)

In [ ]:
#| export
def cardLookup(kind,index):
    if kind == 1:
        return people[index]
    if kind == 2:
        return weapons[index]
    return rooms[index]

def keyCard(kind,index):
    return kind*100 + index

def keyLookUp(key):
    kind = key // 100
    index = key % 100
   # print(key,kind,index)
    return cardLookup(kind,index)
     

In [ ]:
#| hide
cCheck = [keyLookUp(x) for x in [306, 102, 204, 300, 308]]
print(cCheck)

[['Lounge', 'grey', '', 'Lounge'], [' Mrs. White', 'white', '♡', ' White'], ['Candlestick', 'cyan', '🕯️', 'Candle'], ['Ball Room', 'grey', '', 'Ball'], ['Study', 'grey', '', 'Study']]


In [ ]:
#| export

def createClue(playerCount):
    bhand = [[1,randint(0,len(people)-1)],[2,randint(0,len(weapons)-1)],[3,randint(0,len(rooms)-1)]]
    ahand = [keyCard(x[0],x[1]) for x in bhand]
    ploc = sample(range(len(rooms)),len(people))
    
    person = ahand[0]
    
    pdeck = [keyCard(1,x) for x in range(len(people))]
    deck = [[1,x] for x in range(len(people)) if x != person]
    deck = [x for x in pdeck if x != person]
    
    weapon = ahand[1]
    wdeck = [keyCard(2,x) for x in range(len(weapons))]
    deck += [x for x in wdeck if x != weapon]
   
    room = ahand[2]
    wdeck = [keyCard(3,x) for x in range(len(rooms))]
    deck += [x for x in wdeck if x != room]

    
    shuffle(deck)
    decks = [ahand]
    
    for i in range(playerCount):
        decks.append([])
        
   

    cIndex = 1
    for card in deck:
        decks[cIndex].append(card)
        cIndex += 1
        if cIndex > playerCount:
            cIndex = 1
    decks.append(ploc)        
    return decks


def printPlayer(game,i):
    ploc = game[-1]
    person = people[i-1]
    d = cardFull(person)
    loc = cardFull(cardLookup(3,ploc[i-1]))
    d = "[" + str(i) + "] " + d  + " is in " + loc 
    print("\n" + d.center(70," "))
 
    ahand = game[i]
    maps = [keyLookUp(x) for x in ahand]
    printHand(maps,cardFull)

    
def printGame(game):
    print("Solution")
    ahand = game[0]
   
    maps = [keyLookUp(x) for x in ahand]
    printHand(maps,cardFull)
    for i in range(1,len(game)-1):
        printPlayer(game,i)
        

def printLocs(game):

    ploc = game[-1]
    #print(ploc)
    for i in range(len(ploc)):
        person = people[i]
        d = cardFull(person)
        loc = cardFull(cardLookup(3,ploc[i-1]))
        d = "[" + str(i+1) + "] " + d  + " is in " + loc 
        print(d)
     
       

In [ ]:
#| hide
#agame = createClue(4)
amoves = []
printGame(agame)

Solution
🦚 Mrs. Peacock, 🪢 Rope, Lounge

          [1] 👠 Miss Scarlet is in Library          
🔫 Revolver, 🔪 Knife, Hall, 🐉 Mr. Green, Ball Room

        [2] 🧐 Colonel Mustard is in Library         
Kitchen, Study, Conservatory, 🧐 Colonel Mustard, Billiard Room

       [3] ♡  Mrs. White is in Billiard Room        
🔧 Wrench, Dining Room, ♡  Mrs. White, Library

            [4] 🐉 Mr. Green is in Lounge            
🪠 Lead Pipe, 👠 Miss Scarlet, 🍆 Professor Plum, 🕯️ Candlestick


In [ ]:
agame

In [ ]:
#| export
def getSeen(playerID, game, moves):
    foo = [x[-1] for x in moves if x[3] == playerID]
    return game[playerID] + foo


def location(playerID,game):
    ploc = game[-1]
    return ploc[playerID-2]
    

In [ ]:
#| export
def getPlay(pc,game, moves):
    
    ploc = game[-1]
    
    r2 = location(pc,game)
    
    person = people[pc-1]
    d = cardFull(person)
    #print(pc,r2)
    loc = cardFull(cardLookup(3,r2))
    
    d = "[" + str(pc) + "] " + d  + " is in " + loc 
    print("\n" + d.center(70," "))

    ahand = game[pc]
    maps = [keyLookUp(x) for x in ahand]
    #printHand(maps,cardFull)

    seen = [keyLookUp(x)[0] for x in getSeen(pc,game, moves)]
    exclude  = set(seen)
    

    print("\nSuspect")
    p = chooseCard(people,cardFull,flag=exclude)-1
    print("\n\nWeapon")
    w = chooseCard(weapons,cardFull,flag=exclude)-1
    

    print("\n\nMove To")
    passages = trips(r2,len(rooms))

                     
    locs = [cardLookup(3,x) for x in passages ]
    l = chooseCard(locs,cardFull,flag=exclude) 
    l = trips(r2,len(rooms))[l-1]
    
    return (p,w,l)



    

def trips(x,l):
    r1 = x -1
    if r1 < 0:
        r1 = l - 1
    r3 = x + 1
    if r3 > l - 1:
        r3 = 0
    return [r1,x,r3]



In [ ]:
#| export

def showChoice(p,w,l):
    ret = cardFull(cardLookup(1,p))
    ret = ret + " with the " + cardFull(cardLookup(2,w))
    ret = ret + " in the " + cardFull(cardLookup(3,l))
    return ret


def keysForGuess(p,w,l):
    h = [[1,p],[2,w],[3,l]]
    h = [keyCard(x[0],x[1]) for x in h]
    return h

def getAnswer(playerNum,game,guess):
    #print(guess)
    (p,w,l) = guess
    #print(showChoice(p,w,l))
    h = keysForGuess(p,w,l)
    
   
    shuffle(h)
    
    playCount = len(game) - 1
    
    c = playerNum+1
    
    if c > playCount:
        c = 1
    while c != playerNum:
        hand = game[c]
        #printPlayer(game,c)
        for y in h:
            for x in hand:
                #print(x,y,cardFull(keyLookUp(x)),cardFull(keyLookUp(y)))
                if x == y :
                    return (p,w,l,playerNum, c,y)
        #print(str(c) + " pass")
        c = c+1
        if c >= playCount:
            c = 1
    return (-1,0)
    

In [ ]:
#| hide
print(agame[-1])
print(agame)

[5, 8, 3, 7, 2, 1]
[[105, 202, 300], [302, 306, 104, 305, 307], [205, 101, 303, 102, 301], [204, 308, 200, 103, 304], [300, 100, 201, 203], [5, 8, 3, 7, 2, 1]]


In [ ]:
#| hide
agame = createClue(4)
amoves = []
player = 1
done = False
printGame(agame) 
print()
printLocs(agame) 

Solution
🦚 Mrs. Peacock, 🪢 Rope, Lounge

           [1] 👠 Miss Scarlet is in Study           
🔫 Revolver, 🔪 Knife, Hall, 🐉 Mr. Green, Ball Room

        [2] 🧐 Colonel Mustard is in Kitchen         
Kitchen, Study, Conservatory, 🧐 Colonel Mustard, Billiard Room

       [3] ♡  Mrs. White is in Billiard Room        
🔧 Wrench, Dining Room, ♡  Mrs. White, Library

             [4] 🐉 Mr. Green is in Hall             
🪠 Lead Pipe, 👠 Miss Scarlet, 🍆 Professor Plum, 🕯️ Candlestick

[1] 👠 Miss Scarlet is in Library
[2] 🧐 Colonel Mustard is in Study
[3] ♡  Mrs. White is in Kitchen
[4] 🐉 Mr. Green is in Billiard Room
[5] 🍆 Professor Plum is in Hall
[6] 🦚 Mrs. Peacock is in Dining Room


In [ ]:
#| hide
ans = getPlay(player,agame,amoves)
ans


          [1] 👠 Miss Scarlet is in Library          

Suspect
1)  👠 Miss Scarlet
2)  🧐 Colonel Mustard
3)  ♡  Mrs. White
4)* 🐉 Mr. Green
5)  🍆 Professor Plum
6)  🦚 Mrs. Peacock


Choose from 1 to 6:  2




Weapon
1)* 🔪 Knife
2)* 🔫 Revolver
3)  🪢 Rope
4)  🔧 Wrench
5)  🕯️ Candlestick
6)  🪠 Lead Pipe


Choose from 1 to 6:  3




Move To
1)  Lounge
2)  Library
3)  Study


Choose from 1 to 3:  3


(1, 2, 8)

In [ ]:


   
 
    
struct ClueItems:

    let weapons: [[String]]
    
    fn __init__(inout self):
    
        var wp: [[String]] = []
        
        wp.append(["Knife","cyan",u"\U0001F52A","Knife"])
        wp.append(["Revolver","cyan",u"\U0001F52B","Gun"])
        wp.append(["Rope","cyan",'🪢',"Rope"])
        wp.append(["Wrench","cyan",'🔧',"Wrench"])
        wp.append(["Candlestick","cyan",'🕯️',"Candle"])
        wp.append(["Lead Pipe","cyan",'🪠',"Pipe"])
        
        self.weapons = wp


def main():
    print("hello world")
    for x in range(9, 0, -3):
        print(x)
    let items = ClueItems()
    #print(items

([[103, 203, 307],
  [306, 301, 102, 304, 205],
  [105, 201, 303, 307, 204],
  [104, 305, 100, 308, 302],
  [200, 202, 101, 300],
  [1, 3, 2, 7, 2, 2]],
 (3, 3, 2))

In [ ]:

#| export
def relocatePlayers(playerNum , p,l,game):
  
    game[-1][playerNum-2] = l
    game[-1][p-1] = l
    #print(playerNum-1,p)
    #printLocs(game)
    return  game

In [ ]:
#| hide
relocatePlayers(player , ans[0],ans[2],agame)
agame, ans

([[105, 202, 306],
  [201, 200, 304, 103, 300],
  [305, 308, 302, 101, 301],
  [203, 303, 102, 307],
  [205, 100, 104, 204],
  [8, 5, 1, 4, 3, 8]],
 (1, 2, 8))

In [ ]:
#| hide
agame = createClue(4)
amoves = []
player = 1
done = False
printLocs(agame) 

[1] 👠 Miss Scarlet is in Hall
[2] 🧐 Colonel Mustard is in Dining Room
[3] ♡  Mrs. White is in Kitchen
[4] 🐉 Mr. Green is in Conservatory
[5] 🍆 Professor Plum is in Study
[6] 🦚 Mrs. Peacock is in Library


In [ ]:
#| hide
done = False


sh = agame[1]

printHand([keyLookUp(x) for x in sh],cardFull)

while not done:
    
    ans = getPlay(player,agame,amoves)
    agame = relocatePlayers(player , ans[0],ans[2],agame)
    print("\nPlayer " + str(player) + " suspects " + showChoice(ans[0],ans[1],ans[2]))
    check = getAnswer(player,agame,ans)
    if check[0] < 0:
        seen =  getSeen(player,agame, amoves)
        exclude  = set(seen)
        done = True
        for x in keysForGuess(ans[0],ans[1],ans[2]):
            if x in exclude:
                print("\n\n gave a false one " +  cardFull(keyLookUp(x)))
                done = False
        
    else:
        #(p,w,l,playerNum, c,y[0],y[1])
        card = check[-1]
        
        amoves.append(check)
        print("\n\nPlayer " + str(check[-2]) + " has " + cardFull(keyLookUp(card)))
        #printLocs(agame)
       # player = player + 1
        if player>len(agame)-1:
           player = 1 

print("Player " + str(player) + " won")
printGame(agame) 


Hall, 🕯️ Candlestick, 🪠 Lead Pipe, Lounge, 🦚 Mrs. Peacock

        [1] 👠 Miss Scarlet is in Dining Room        

Suspect
1)  👠 Miss Scarlet
2)* 🧐 Colonel Mustard
3)* ♡  Mrs. White
4)* 🐉 Mr. Green
5)* 🍆 Professor Plum
6)* 🦚 Mrs. Peacock


Choose from 1 to 6:  2




Weapon
1)* 🔪 Knife
2)* 🔫 Revolver
3)* 🪢 Rope
4)  🔧 Wrench
5)* 🕯️ Candlestick
6)* 🪠 Lead Pipe


Choose from 1 to 6:  4




Move To
1)  Conservatory
2)  Dining Room
3)* Hall


...



Weapon
1)* 🔪 Knife
2)* 🔫 Revolver
3)* 🪢 Rope
4)  🔧 Wrench
5)* 🕯️ Candlestick
6)* 🪠 Lead Pipe


Choose from 1 to 6:  4




Move To
1)  Conservatory
2)  Dining Room
3)* Hall


Choose from 1 to 3:  2



Player 1 suspects 👠 Miss Scarlet with the 🔧 Wrench in the Dining Room
Player 1 won
Solution
👠 Miss Scarlet, 🔧 Wrench, Dining Room

        [1] 👠 Miss Scarlet is in Dining Room        
Hall, 🕯️ Candlestick, 🪠 Lead Pipe, Lounge, 🦚 Mrs. Peacock

      [2] 🧐 Colonel Mustard is in Dining Room       
🍆 Professor Plum, 🪢 Rope, Billiard Room, Ball Room, 🔫 Revolver

        [3] ♡  Mrs. White is in Dining Room         
🐉 Mr. Green, Kitchen, 🔪 Knife, ♡  Mrs. White

         [4] 🐉 Mr. Green is in Dining Room          
Conservatory, 🧐 Colonel Mustard, Library, Study


In [ ]:
#| hide
printGame(agame) 

Solution
🦚 Mrs. Peacock, 🪢 Rope, Lounge

          [1] 👠 Miss Scarlet is in Library          
🔫 Revolver, 🔪 Knife, Hall, 🐉 Mr. Green, Ball Room

        [2] 🧐 Colonel Mustard is in Library         
Kitchen, Study, Conservatory, 🧐 Colonel Mustard, Billiard Room

       [3] ♡  Mrs. White is in Billiard Room        
🔧 Wrench, Dining Room, ♡  Mrs. White, Library

            [4] 🐉 Mr. Green is in Lounge            
🪠 Lead Pipe, 👠 Miss Scarlet, 🍆 Professor Plum, 🕯️ Candlestick


In [ ]:
#| hide
chooseCard(rooms,cardFull)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()